# Exploratory Data Analysis

**Ziele**:
- Daten verstehen
- Eventuelle Probleme identifizieren

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer


import re
import os
from collections import Counter

nltk.download('punkt')
nltk.download('stopwords')
init_notebook_mode(connected=True)
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = [20, 8]
plt.rcParams['font.size'] = 18 

## Herunterladen der Datasets
Wir nutzen die Kaggle API um zwei Datensaetze herunterzuladen. Damit die Authentifizierung ohne Probleme funktioniert, muss zunaechst ein Kaggle Account angelegt werden und anschliessend ein enstprechender Access Token generiert werden. 

Datensaetze:
- McDonalds Reviews
- IMDB Reviews
- (Amazon Food Reviews)

**Wichtig:** Falls Google Colab genutzt wird, muss die `kaggle.json` (enthaelt die API credentials) hochgeladen werden. Andernfalls kann der naechste Schritt ubersprungen werden.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download McDonalds dataset
if not os.path.exists('mcdonalds-store-reviews.zip'):
    print("Downloading McDonalds dataset...")
    !kaggle datasets download -d nelgiriyewithana/mcdonalds-store-reviews
if os.path.exists('mcdonalds-store-reviews.zip'):
    print("Unzipping McDonalds dataset...")
    !unzip -n mcdonalds-store-reviews.zip

# Download IMDB dataset
if not os.path.exists('imdb-dataset-of-50k-movie-reviews.zip'):
    print("Downloading IMDB dataset...")
    !kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
if os.path.exists('imdb-dataset-of-50k-movie-reviews.zip'):
    print("Unzipping IMDB dataset...")
    !unzip -n imdb-dataset-of-50k-movie-reviews.zip

### Zusaetzliches Datenset: Amazon Fine Food Reviews

In [ ]:
# Zusaetzliches Amazon Review Daa
if not os.path.exists('amazon-fine-food-reviews.zip'):  
    print("Downloading Amazon dataset...")
    !kaggle datasets download -d snap/amazon-fine-food-reviews
if os.path.exists('amazon-fine-food-reviews.zip'):  
    print("Unzipping Amazon dataset...")
    !unzip -n amazon-fine-food-reviews.zip

## Laden der Datensaetze

In [ ]:
df_mcd = pd.read_csv('McDonald_s_Reviews.csv', encoding='latin-1')
df_imdb = pd.read_csv('IMDB Dataset.csv')
df_amazon = pd.read_csv('Reviews.csv')

## Erster Ueberblick 

**McDonalds Reviews**

In [ ]:
df_mcd.head()   

**IMDB Reviews**

In [ ]:
df_imdb.head()

**(Amazon Fine Food Reviews)**

In [ ]:
df_amazon.head()

Neben Reviewtexten und Bewertungen sind in den Datensaetzen einige weitere Daten enthalten. Da wir folgend nur an den Textdaten und dessen Bewertung interessiert sind, koennen wir diese herausziehen.

In [ ]:
# Only keep the text and the sentiment
df_mcd = df_mcd[['review', 'rating']]
df_imdb = df_imdb[['review', 'sentiment']]  
df_amazon = df_amazon[['Text', 'Score']]

Ebenfalls koennen wir sehen, dass die Label 'categorical' sind und in form eines strings vorliegen. Um die Label besser verwenden zu koennen werden diese zunaechst in ein numerisches Format umgewandelt. Fuer das Imdb Dataset wird `positive` zu `1` und `negative` zu `0`. Selbes bei den McDonald Reviews, `1 star` entspricht `1` und `5 stars` entspricht `5`. (Im Fall des Amazon Datasets entspricht der 'Score' bereits einem integer Datentypen.)

Grundsaetzlich laesst sich bereits sagen, dass die Klassifizierungsprobleme hier einmal eine 'Binary Classification' und einmal eine 'Multi-Class Classification' sind. Fuer die Visualiserung ist dieses zunaechst kein Problem, allerdings muss bei einer spaeteren Zusammenfuerhung beachtet werden inwiefern die kategorialen daten entweder zu positv (1) oder negativ (0) umgewandelt werden.

In [ ]:
mcd_rating_map = {
    '1 star': 1,
    '2 stars': 2,
    '3 stars': 3,
    '4 stars': 4,
    '5 stars': 5,
}
df_mcd['rating'] = df_mcd['rating'].map(mcd_rating_map)
df_imdb['sentiment'] = df_imdb['sentiment'].map({'positive': 1, 'negative': 0})

# Align column names
df_mcd = df_mcd.rename(columns={'review': 'text', 'rating': 'sentiment'})
df_amazon = df_amazon.rename(columns={'Text': 'text', 'Score': 'sentiment'})
df_imdb = df_imdb.rename(columns={'review': 'text', 'sentiment': 'sentiment'})

Das McDonalds und Amazon Dataset haben nachwievor kategorial gelabelte Reviews 1-5, das IMDB Dataset hingegen hat nur binary label mit 0-1. 

In [ ]:
df_mcd.head()   

In [ ]:
df_mcd.info()

In [ ]:
df_imdb.head(5) 

In [ ]:
df_imdb.info()

In [ ]:
df_amazon.head()

In [ ]:
df_amazon.info()

### Anzal der Eintraege

In [ ]:
print("McDonalds dataset has {} entries".format(len(df_mcd)))
for i in range(1, 6):
    print("McDonalds dataset has {} entries with {} stars".format(len(df_mcd[df_mcd['sentiment'] == i]), i))

print('-' * 50)

print("IMDB dataset has {} entries".format(len(df_imdb)))
for i in range(0, 2):
    print("IMDB dataset has {} entries with {} sentiment".format(len(df_imdb[df_imdb['sentiment'] == i]), 'positive' if i == 1 else 'negative'))

print('-' * 50)

print("Amazon dataset has {} entries".format(len(df_amazon)))
for i in range(1, 6):
    print("Amazon dataset has {} entries with {} stars".format(len(df_amazon[df_amazon['sentiment'] == i]), i))


## Rating / Sentiment Verteilung

In [ ]:
sns.countplot(x='sentiment', data=df_mcd)
plt.title('McDonalds Rating Distribution')
plt.xlabel('Rating')
plt.show()

In [ ]:
# IMDB Sentiment Distribution
sns.countplot(x='sentiment', data=df_imdb)
plt.title('IMDB Sentiment Distribution')    
plt.show()

In [ ]:
# Amazon Rating Distribution
sns.countplot(x='sentiment', data=df_amazon)
plt.title('Amazon Rating Distribution')
plt.xlabel('Rating')
plt.show()

## Genauere Betrachtung das McDonalds Datasets

- Charakteranzahl pro Kategorie
- Wortanzahl pro Kategorie 
- Durschnittliche Wortlaenge pro Kategorie
- Durchschnittliche Satzlaenge pro Kategorie 
- Meistvorkommende Woerter 

In [ ]:
df_mcd['char_count'] = df_mcd['text'].apply(len)
df_mcd['word_count'] = df_mcd['text'].apply(lambda x: len(word_tokenize(x)))
df_mcd['mean_word_length'] = df_mcd['text'].apply(lambda x: np.mean([len(word) for word in word_tokenize(x)]))
df_mcd['sent_count'] = df_mcd['text'].apply(lambda x: len(sent_tokenize(x)))
df_mcd['mean_sent_length'] = df_mcd['word_count'] / df_mcd['sent_count']  
df_mcd.head()     

In [ ]:
sns.boxplot(x='sentiment', y='char_count', data=df_mcd)
plt.title('McDonalds Character Count per Review & Category')
plt.xlabel('Rating')
plt.ylabel('Character Count')   
plt.show()

In [ ]:
sns.boxplot(x='sentiment', y='word_count', data=df_mcd) 
plt.title('McDonalds Word Count per Review & Category')
plt.xlabel('Rating')
plt.ylabel('Word Count')
plt.show()

In [ ]:
# sentence cound per review
sns.boxplot(x='sentiment', y='sent_count', data=df_mcd)
plt.title('McDonalds Sentence Count per Review & Category')
plt.xlabel('Rating')
plt.ylabel('Sentence Count')
plt.show()

In [ ]:
df_mean_lengths = df_mcd.groupby('sentiment').agg({'mean_word_length': 'mean', 'mean_sent_length': 'mean'}).reset_index()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

# Mean Word length
sns.barplot(x='sentiment', y='mean_word_length', data=df_mean_lengths, ax=axs[0])
axs[0].set_title('Mean Word Length per Rating Category')
axs[0].set_xlabel('Rating')
axs[0].set_ylabel('Mean Word Length')

# Mean Sentence Length 
sns.barplot(x='sentiment', y='mean_sent_length', data=df_mean_lengths, ax=axs[1])
axs[1].set_title('Mean Sentence Length per Rating Category')
axs[1].set_xlabel('Rating')
axs[1].set_ylabel('Mean Sentence Length')

plt.subplots_adjust(wspace=0.3)
plt.tight_layout()
plt.show()

Die vorhergehenden Plots zeigen relativ Interessante Dinge auf. Generell kann gesagt werden, dass schlechte Bewertungen tendenziell laenger sind und mehr Saetze enthalten. Dies deutet darauf hin, dass unzufriedene Kunden detailierteres Feedback geben. 

In [ ]:
# Clean up added columns
df_mcd.drop(['char_count', 'word_count', 'mean_word_length', 'sent_count', 'mean_sent_length'], axis=1, inplace=True)
df_mcd.head()

## Haeufigkeit von Begriffen
Bevor wir die Haeufigkeit von Begriffen analysieren, ist es notwendig, die Datensaetze zu bereinigen, um valide Ergebnisse zu erzielen. Es ist wichtig, irrelevante Daten wie HTML-Tags und Zeichensetzung zu entfernen, da diese in der folgenden Analyse zu Verzerrungen fuehren koennen. Durch die Umwandlung aller Woerter in Kleinbuchstaben erreichen wir eine gewisse Standardisierung, die uns konsistentere Resultate liefert. Zudem erlaubt das Entfernen von Stoppwoertern, die ueblicherweise wenig zur Information beitragen, die Konzentration auf wirklich relevante Begriffe. Dies stellt sicher, dass unsere Analyse praezise und informativ ist.

### Reinigen der Reviews

In [ ]:
def clean_text(text):
    # Convert to lowercase
    test = text.lower()
    # Remove punctuation
    test = re.sub(r'[^a-z\s]', '', test)
    # Remove HTML Tags
    test = re.sub(r'<.*?>', '', test)
    # Remove stopwords
    test = ' '.join([word for word in word_tokenize(test) if word not in stopwords.words('english')])
    return test

Was hier sicherlich auch noch interessant waere ist 'Stemming' und 'Lemmatization'.
- Stemming: Reduziert Woerter auf ihre Basis, z.B. aus 'jumps', 'jumping', 'jumped' wird 'jump'
- Lemmatization: Reduziert Woerter auf ihre lexikalische Grundform, bietet hoehere semantische Genauigkeit als Stemming. 

In [ ]:
og_text = df_mcd['text'][0]
df_mcd['text'] = df_mcd['text'].apply(clean_text)

print('Original Text: {}'.format(og_text))
print('Cleaned Text: {}'.format(df_mcd['text'][0]))

In [ ]:
df_mcd.head()

### Anlegen eines Wort - Haeufigkeits Mappings (Aehnlich BoW)

In [ ]:
df_mcd['word_list'] = df_mcd['text'].apply(lambda x: x.split())
df_mcd.head()

In [ ]:
# Create a list of all words and the frequency of each word
word_list = []
for index, row in df_mcd.iterrows():
    word_list += row['word_list']

print('Total Words: {}'.format(len(word_list)))
most_common_words = Counter(word_list).most_common(10)
print('Most Common Words: {}'.format(most_common_words))

# Could also use nltk.FreqDist
# word_freq = nltk.FreqDist(word_list)

In [ ]:
# Create a list for all words and a list for the frequencies
words = [word for word,_ in most_common_words]
freqs = [freq for _,freq in most_common_words]

# plot with seaborn
sns.barplot(x=freqs, y=words)
plt.title('Top 10 Most Common Words')
plt.show()

### Haeufigkeit von Bi-grammen
Woerter fuer sich alleine geben meistens nicht den gesamten Kontext wieder. Abhaengig von der Position im Satz koennen sie sogar komplett unterschiedliche Bedeutungen haben. Mit der Betrachtung der Bi-Grams koennen wir ein besseres Verstaendnis fuer den Kontext bekommen, in dem Worte verwendet werden. Des Weiteren hilft diese Analyse gaengige Phrasen und Ausdruecke zu identifizieren, welche haeufig in Reviews verwendet werden, z.B. 'good service', 'cold food' etc.

In [ ]:
cv = CountVectorizer(ngram_range=(2,2))
bigrams = cv.fit_transform(df_mcd['text'])

print('Total Bigrams: {}'.format(bigrams.shape[1]))
print('Shape: {}'.format(bigrams.shape))       

In [ ]:
count_values = bigrams.toarray().sum(axis=0)
ngram_freq = pd.DataFrame(sorted([(count_values[i], k) for k, i in cv.vocabulary_.items()], reverse = True))
ngram_freq.columns = ["frequency", "ngram"]

In [ ]:
sns.barplot(x=ngram_freq['frequency'][:10], y=ngram_freq['ngram'][:10])
plt.title('Top 10 Most Common Bigrams')
plt.show()

Bis auf 'worst mcdonalds' scheinen die Bi-Grams entweder positiv, oder nichts-aussagend zu sein. Hier fehlt uns oft ein weiteres Wort um z.B. bei 'customer service', um eine Bewertung ueber das Sentiment machen zu koennen. Hier waere es interessant zu schauen inwiefern sich die haeufigsten Woerter und BiGramme zwischen negativen und postiven reviews unterscheiden.

### Auftrennung in 'positive' und 'negative' reviews

Da die Labels des McDonalds Datensatzes 'multiclass' sind, muss zunaechst eine Entscheidung getroffen werden, welche Reviews 'postiv' und welche 'negativ' sind. Der Einfachheit halber haben wir uns fuer folgende Aufteilung entschieden:
- Ratings <= 3 sind negativ
- Ratings > 3 sind positiv

In [ ]:
df_mcd['sentiment'] = df_mcd['sentiment'].map(lambda x: 1 if x > 3 else 0)  
df_mcd.head()

In [ ]:
negative_reviews = df_mcd[df_mcd['sentiment'] == 0]
positive_reviews = df_mcd[df_mcd['sentiment'] == 1]

print('Number of Negative Reviews: {}'.format(len(negative_reviews)))
print('Number of Positive Reviews: {}'.format(len(positive_reviews)))

In [ ]:
# Create a list of all words and the frequency of each word for negative reviews and positive reviews
negative_word_list = []
for index, row in negative_reviews.iterrows():
    negative_word_list += row['word_list']

most_common_negative_words = Counter(negative_word_list).most_common(10)
neg_freqs = [freq for _,freq in most_common_negative_words]     
neg_words = [word for word,_ in most_common_negative_words]

positive_word_list = []
for index, row in positive_reviews.iterrows():
    positive_word_list += row['word_list']

most_common_positive_words = Counter(positive_word_list).most_common(10)
pos_freqs = [freq for _,freq in most_common_positive_words]
pos_words = [word for word,_ in most_common_positive_words]

In [ ]:
# plot with seaborn
sns.barplot(x=pos_freqs, y=pos_words)   
plt.title('Top 10 Most Common Positive Words')
plt.xlabel('frequency')
plt.show()

In [ ]:
sns.barplot(x=neg_freqs, y=neg_words)
plt.title('Top 10 Most Common Negative Words')
plt.show()

In [ ]:
cv_neg = CountVectorizer(ngram_range=(2,2))
bigrams_neg = cv_neg.fit_transform(negative_reviews['text'])

print('Negative - Total Bigrams: {}'.format(bigrams_neg.shape[1]))
print('Shape: {}'.format(bigrams_neg.shape))   

cv_pos = CountVectorizer(ngram_range=(2,2))
bigrams_pos = cv_pos.fit_transform(positive_reviews['text'])

print('Positive - Total Bigrams: {}'.format(bigrams_pos.shape[1]))
print('Shape: {}'.format(bigrams_pos.shape))    

In [ ]:
count_values_neg = bigrams_neg.toarray().sum(axis=0)
ngram_freq_neg = pd.DataFrame(sorted([(count_values[i], k) for k, i in cv_neg.vocabulary_.items()], reverse = True))
ngram_freq_neg.columns = ["frequency", "ngram"]

count_values_pos = bigrams_pos.toarray().sum(axis=0)
ngram_freq_pos = pd.DataFrame(sorted([(count_values[i], k) for k, i in cv_pos.vocabulary_.items()], reverse = True))
ngram_freq_pos.columns = ["frequency", "ngram"]

In [ ]:
sns.barplot(x=ngram_freq_pos['frequency'][:10], y=ngram_freq_pos['ngram'][:10])
plt.title('Top 10 Most Common Positive Bigrams')
plt.show()

In [ ]:
sns.barplot(x=ngram_freq_neg['frequency'][:10], y=ngram_freq_neg['ngram'][:10])
plt.title('Top 10 Most Common Negative Bigrams')
plt.show()